# Optional: downloading training texts

In [1]:
!pip install dtrx

     |████████████████████████████████| 97 kB 6.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=50b2f3c9fc19fb0c1dcdc7dc213fc4b9993e05c1ba477702f5621d8433d8bdc2
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
Successfully built subprocess32


In [ ]:
!lang="vi"; wget -c https://dumps.wikimedia.org/${lang}wiki/latest/${lang}wiki-latest-pages-articles.xml.bz2 & wget https://object.pouta.csc.fi/OPUS-WikiMatrix/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-wikimedia/v20210402/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-CCAligned/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-XLEnt/v1.1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-JW300/v1b/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-TED2020/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-GNOME/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-ParaCrawl/v8/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-TildeMODEL/v2018/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-DGT/v2019/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-EUbookshop/v2/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-SETIMES/v2/mono/${lang}.txt.gz

In [ ]:
!dtrx *.txt.gz.* & dtrx *.txt.gz & dtrx *.bz2

In [ ]:
!cat *.txt > dataset.txt

# Downloading and cleaning Wikipedia

In [ ]:
!nohup curl -L -O https://dumps.wikimedia.org/ltwiki/latest/ltwiki-latest-pages-articles.xml.bz2 &

In [4]:
!pip install wikiextractor


     |████████████████████████████████| 46 kB 1.9 MB/s 


In [53]:
!python3 -m wikiextractor.WikiExtractor -o wiki-lt/ --no-templates --processes 8 ltwiki-latest-pages-articles.xml.bz2

INFO: Starting page extraction from ltwiki-latest-pages-articles.xml.bz2.
INFO: Using 8 extract processes.
INFO: Extracted 100000 articles (430.4 art/s)
INFO: Extracted 200000 articles (558.9 art/s)
INFO: Finished 8-process extraction of 286412 articles in 615.0s (465.7 art/s)


In [ ]:
!du; ls -s

In [58]:
!cat wiki*/*/* > final_untokenised.txt

In [59]:
!cat wiki*/AA/wiki_*0* > tokentrain_final.txt

# Training the tokeniser

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [17]:
!pip install youtokentome & pip install tokenizers

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 4.7 MB/s 


In [18]:
import youtokentome as yttm

In [61]:
yttm.BPE.train('/content/tokentrain_final.txt', 'model', 50000, 0.9999, n_threads=-1, pad_id=0, unk_id=1, bos_id=2, eos_id=3)

In [62]:
!yttm vocab --model model > vocab.txt

In [64]:
!sed 's/^ *[0-9]\+.//g' /content/vocab.txt > /content/cleanvocab.txt

In [65]:
text_file = open("/content/cleanvocab.txt", "r")
lines = text_file.read().split('\n')

In [ ]:
lines = [w.replace('▁', ' ') for w in lines]
print(lines)

# Encoding texts and making a tokeniser.json

In [ ]:
initial = '{"version":"1.0","truncation":null,"padding":null,"added_tokens":[{"id":50256,"special":true,"content":"<|endoftext|>","single_word":false,"lstrip":false,"rstrip":false,"normalized":true}],"normalizer":null,"pre_tokenizer":{"type":"ByteLevel","add_prefix_space":false,"trim_offsets":true},"post_processor":{"type":"ByteLevel","add_prefix_space":true,"trim_offsets":false},"decoder":{"type":"ByteLevel","add_prefix_space":true,"trim_offsets":true},"model":{"dropout":null,"unk_token":null,"continuing_subword_prefix":"","end_of_word_suffix":"","fuse_unk":false,"vocab":{'
i = 0
for token in lines:
    if '\\' in token:
       token = token.replace('\\', '\\\\')
    if '"' in token:
       token = token.replace('"', '\\"')
    initial = initial + '"' + token + '":' + str(i) + ','
    i = i + 1
initial = initial[:-1]
initial = initial + '},"merges":[]}}'
print(initial)

In [68]:
with open("aitextgen.tokeniser.json", "w") as text_file:
    text_file.write(initial)

In [ ]:
!yttm encode --model /content/model --n_threads 8 --dropout_prob 0.25 --output_type id < /content/final_untokenised.txt > final.txt

n_threads: 8
bytes processed: 10485889bytes processed: 20971810bytes processed: 31457605bytes processed: 41943477bytes processed: 52429373bytes processed: 62915143bytes processed: 73400982bytes processed: 83887406bytes processed: 94373436bytes processed: 104859204bytes processed: 115345178bytes processed: 125831568bytes processed: 136317998bytes processed: 146803884bytes processed: 157289724bytes processed: 167776410bytes processed: 178262232bytes processed: 188748023bytes processed: 199233816bytes processed: 

# Saving as npy array

There may be a more efficient way of doing this. I tried using linecache to read specific lines and then multithreaded it but then np.save broke, I'm not sure why.

In [ ]:
!cat /content/final.txt /content/translations_tokenised_new.txt FINAL_ALL.txt

In [ ]:
from pathlib import Path
import numpy as np
import os
import re
file = open("dataset_tokenized.txt")
i = 0
allLines = []
while 1:
    allLines = []
    i = i + 50000000
    string_data = file.read(50000000)
    if not string_data:
        break
    list_data = re.split(' |\n',string_data)
    list_data = list(filter(None, list_data))
    arr = np.array(list_data, dtype=np.uint16)
    f_handle = open('data.npy', 'ab')
    np.save(f_handle, arr)
    f_handle.close()
    print('Line ' + str(i) + ' done!')

In [ ]:
!gzip data.npy

In [ ]:
!cp data.npy.gz /content/drive/MyDrive/